In [9]:
# Start writing code here...

import pandas as pd
  
  
df = pd.read_json("example.json")
df.head()

,transactions
0,"{'description': 'Spotify', 'amount': -14.99, '..."
1,"{'description': 'Spotify', 'amount': -14.99, '..."
2,"{'description': 'Spotify', 'amount': -14.99, '..."
3,"{'description': 'Spotify', 'amount': -14.99, '..."
4,"{'description': 'Netflix', 'amount': -20.0, 'd..."


In [7]:
import json

 
# Opening JSON file
with open('example.json') as json_file:
    data = json.load(json_file)
 

In [92]:
# Just converting data in list to make it easy for preprocessing

def data_to_arrays(data):
    description= []
    amount = []
    date= []


    for transaction in data['transactions']:
        description.append(transaction['description'])
        amount.append(transaction['amount'])
        date.append(transaction['date'])

    return description, amount, date

description, amount, date= data_to_arrays(data)

In [94]:
# Let's do basis preprocessing
# 1. text standardization (capital letters), removing numbers etc 

import re

def convert_string_to_basic(string_init):
    # replace special characters with a space 

    string_init= str(string_init)
    my_new_string = re.sub('[^a-zA-Z0-9 \n\.]', ' ', string_init)

    # replace numbers with nothing 

    my_new_string= re.sub('[0-9]', '', my_new_string)


    # Replace multiple while spaces with a single one

    my_new_string= ' '.join(my_new_string.split())

    my_new_string= my_new_string.lower()
    return my_new_string



new_description= []
for d in description:


    new_d= convert_string_to_basic(d)
    new_description.append(new_d)


In [18]:
# find repeat transactions first - just to visualize and see if there are any other case that we might incorporate

des_key_val= {}

for d in new_description:
    if d in des_key_val.keys():
        des_key_val[d]= des_key_val[d]+ 1
    else:
        des_key_val[d]= 1

des_key_val

{'spotify': 4,
 'netflix': 4,
 'aug acme corp salary': 1,
 'sep acme corp salary': 1,
 'oct acme corp salary': 1,
 'nov acme corp salary': 1,
 'dec acme corp salary': 1,
 'jan acme corp salary': 1,
 'feb acme corp salary': 1,
 'mar acme corp salary': 1,
 'apr acme corp salary': 1,
 'company lunch burritos': 1,
 'company lunch korean': 1,
 'company lunch barbeque': 1,
 'company lunch hot pot': 1,
 'company lunch fondue': 1,
 'company lunch burgers': 1,
 'one off meal': 1,
 'special meal': 1,
 'special holiday': 1,
 'treating myself': 1}

In [ ]:
# NOTE HERE

# Assumption 'aug acme corp salary' and sep acme corp salary' can are recurring 
# We can try fuzzy search to find 1,3 letter difference and then maybe treat enteries with this difference the same. 

# When this won't work?
# For short descriptions
# Will likely have a lot of false positives.

In [30]:
# Let's use regex then to remove dates from descriptions. 
# NOTE: Just the basic case of regex added. This wont work for other dates. NOT ROBUST

def remove_dates(sentence):
    sentence = re.sub('(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)', ' ', sentence)
    return sentence

In [35]:
new_description_regex= []
for d in new_description:
    new_d= remove_dates(d)
    new_description_regex.append(new_d)

In [48]:
# find repeat transactions first
des_key_val= {}

# also group all dates together for recurring names 

index= 0
for d in new_description_regex:
    if d in des_key_val.keys():
        des_key_val[d][0] = des_key_val[d][0]+ 1

        required_date= date[index]
        des_key_val[d][1].append(required_date)
    else:

        required_date= date[index]
        des_key_val[d]= [1,[required_date]]

    index= index+ 1

# sort all the dates 
for k in des_key_val.keys():
    des_key_val[k][1].sort()

    
des_key_val

{'spotify': [4, ['2021-01-29', '2020-12-29', '2020-11-29', '2020-10-29']],
 'netflix': [4, ['2020-02-15', '2020-03-14', '2020-04-16', '2020-05-15']],
 '  acme corp salary': [9,
  ['2020-08-01',
   '2020-09-01',
   '2020-10-01',
   '2020-11-01',
   '2020-12-01',
   '2021-01-01',
   '2021-02-01',
   '2021-03-01',
   '2021-04-01']],
 'company lunch burritos': [1, ['2021-01-05']],
 'company lunch korean': [1, ['2021-01-12']],
 'company lunch barbeque': [1, ['2021-01-19']],
 'company lunch hot pot': [1, ['2021-01-26']],
 'company lunch fondue': [1, ['2021-02-02']],
 'company lunch burgers': [1, ['2021-02-09']],
 'one off meal': [1, ['2021-02-14']],
 'special meal': [1, ['2021-01-05']],
 'special holiday': [1, ['2020-12-10']],
 'treating myself': [1, ['2020-11-05']]}

In [40]:
# convert date string to datetime 

# arrange dates in assending order

# check if the different between concurrent dates is just 1 month

'2021-01-29'

In [88]:
from dateutil import relativedelta
import datetime


# this function can find the difference between two dates 

def differnce_two_dates(d1, d2):
    date_time_str = d1
    start_date = datetime.datetime.strptime(date_time_str, '%Y-%m-%d')

    date_time_str = d2
    end_date = datetime.datetime.strptime(date_time_str, '%Y-%m-%d')

    # Get the relativedelta between two dates
    delta = relativedelta.relativedelta(end_date, start_date)
    print(delta.years, 'Years,', delta.months, 'months,', delta.days, 'days')

    if delta.years ==0 and delta.months==0:
        if delta.days>=28:
            return True
        else:
            return False

    elif delta.years ==0 and delta.months==1:
        if delta.days<=3:
            return True
        else:
            return False


In [91]:
recurring_transactions= []
potential_recurring = ""

for k in des_key_val.keys():
    if des_key_val[k][0]>1:
        potential_recurring=k

        # now check the difference between dates

        dates= des_key_val[k][1]

        for i in range(len(dates)-1):
            d1= dates[i]
            d2= dates[i+1]
            
            diff= differnce_two_dates(d1, d2)
            if diff== False:
                potential_recurring= ""
    if potential_recurring!= "":
        recurring_transactions.append(potential_recurring)
        potential_recurring = ""

recurring_transactions




0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 0 months, 28 days
0 Years, 1 months, 2 days
0 Years, 0 months, 29 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days


['spotify', 'netflix', '  acme corp salary']

# Compiled Code

In [95]:
# Lets compile the code in one place

# all supoorting functions 

# Just converting data in list to make it easy for preprocessing

def data_to_arrays(data):
    description= []
    amount = []
    date= []


    for transaction in data['transactions']:
        description.append(transaction['description'])
        amount.append(transaction['amount'])
        date.append(transaction['date'])

    return description, amount, date



def convert_string_to_basic(string_init):
    # replace special characters with a space 

    string_init= str(string_init)
    my_new_string = re.sub('[^a-zA-Z0-9 \n\.]', ' ', string_init)

    # replace numbers with nothing 

    my_new_string= re.sub('[0-9]', '', my_new_string)


    # Replace multiple while spaces with a single one

    my_new_string= ' '.join(my_new_string.split())

    my_new_string= my_new_string.lower()
    return my_new_string



# Let's use regex then to remove dates from descriptions. 
# NOTE: Just the basic case of regex added. This wont work for other dates. NOT ROBUST

def remove_dates(sentence):
    sentence = re.sub('(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)', ' ', sentence)
    return sentence


# this function can find the difference between two dates 

def differnce_two_dates(d1, d2):
    date_time_str = d1
    start_date = datetime.datetime.strptime(date_time_str, '%Y-%m-%d')

    date_time_str = d2
    end_date = datetime.datetime.strptime(date_time_str, '%Y-%m-%d')

    # Get the relativedelta between two dates
    delta = relativedelta.relativedelta(end_date, start_date)
    print(delta.years, 'Years,', delta.months, 'months,', delta.days, 'days')

    if delta.years ==0 and delta.months==0:
        if delta.days>=28:
            return True
        else:
            return False

    elif delta.years ==0 and delta.months==1:
        if delta.days<=3:
            return True
        else:
            return False
            


In [98]:
# Let's compile code in one place

# Start writing code here...

import pandas as pd
import json
import re
from dateutil import relativedelta
import datetime

def find_recurring_transactios(file_path):



    
    # Opening JSON file
    with open(file_path) as json_file:
        data = json.load(json_file)
    


    description, amount, date= data_to_arrays(data)



    new_description= []
    for d in description:

        new_d= convert_string_to_basic(d)
        new_description.append(new_d)



    new_description_regex= []
    for d in new_description:
        new_d= remove_dates(d)
        new_description_regex.append(new_d)




    # find repeat transactions first
    des_key_val= {}

    # also group all dates together for recurring names 

    index= 0
    for d in new_description_regex:
        if d in des_key_val.keys():
            des_key_val[d][0] = des_key_val[d][0]+ 1

            required_date= date[index]
            des_key_val[d][1].append(required_date)
        else:

            required_date= date[index]
            des_key_val[d]= [1,[required_date]]

        index= index+ 1

    # sort all the dates 
    for k in des_key_val.keys():
        des_key_val[k][1].sort()

    




    recurring_transactions= []
    potential_recurring = ""

    for k in des_key_val.keys():
        if des_key_val[k][0]>1:
            potential_recurring=k

            # now check the difference between dates

            dates= des_key_val[k][1]

            for i in range(len(dates)-1):
                d1= dates[i]
                d2= dates[i+1]
                
                diff= differnce_two_dates(d1, d2)
                if diff== False:
                    potential_recurring= ""
        if potential_recurring!= "":
            recurring_transactions.append(potential_recurring)
            potential_recurring = ""
            

    return recurring_transactions



In [100]:
find_recurring_transactios("example.json")

0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 0 months, 28 days
0 Years, 1 months, 2 days
0 Years, 0 months, 29 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days
0 Years, 1 months, 0 days


['spotify', 'netflix', '  acme corp salary']

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1d43a8f4-e120-4eb5-9830-0858cdd19102' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>